# 건설자재 품질검사 등록정보

In [1]:
from common import commonFunc as cf
import pandas as pd
import threading

print("실행 금지")
exit()





pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")

SITENAME = "건설사업정보시스템"
DATANAME= "건설자재 품질검사 등록정보"
APIKEY = "1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF"

targetData = metadata.loc[metadata.자료명==DATANAME]
preSetFolder = targetData["저장폴더"].values[0]

preSetFolder

'../output/건설사업정보시스템/건설자재 품질검사 등록정보/selectIoPmQtlTsitStsList.csv'

In [ ]:
lock = threading.Lock()

In [2]:
targetData = metadata.loc[metadata.자료명==DATANAME]

##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
REQPARAM = targetData["요청변수"].values[0]
REQPARAM = REQPARAM.split(",")
PRIMARYKEY = targetData["기본키"].values[0]

In [3]:
JSONKEY = "items"
DUMMY = 0

In [4]:
#### 페이지번호 설정 ###
PAGEYN=1
if REQPARAM.count("pageyn") == 0:
    PAGEYN = 0
else:
    PAGEYN = 1

In [5]:
resultDfMerged = pd.DataFrame()

In [6]:
import datetime

currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
endYr = date.strftime("%Y")
endYr

'2022'

In [7]:
# 기준년도 산정
# 2001년 부터 데이터 존재
startYr = "2001"
dateRange = list(map(int,pd.period_range(startYr,endYr,freq='Y').strftime('%Y').tolist()))
# dateRange

## 건설자재 품질검사 등록정보 전용 스크랩 함수

In [8]:
import requests
from lxml import html
from bs4 import BeautifulSoup 
import pandas as pd
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import os

STDENCODING="utf-8"
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
### 파마리터정의: 
###   - inurl: 메타정보의 "URL"컬럼값 (예: https://www.calspia.go.kr/io/openapi/cm/selectIoCmConstructionList.do )
###   - inSiteName: 메타정보의 "자료대상" (예: 건설사업정보시스템)
###   - inDataName: 메타정보의 "자료명" (예: 공사정보 목록)
###   - inServiceName: 메타정보의 "기본키" (예: serviceKey + Format)
###   - inParam: 메타정보의 "파라미터 정보" (예: 페이지 파라미터 존재 시 1 값")
###   - inPageYn: 메타정보의 "페이지 정보" (예: 페이지 파라미터 존재 시 1 값")
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
def myscrapy(inUrl, inSiteName, inDataName, inServiceName, inParam, inPageYn, jsonkey="items", dummy=0, inType="jsonabnormal",x=-1):
    try:
        emptyPd = pd.DataFrame()
        i=1
        while True:
            print("{} page scraping start".format(i))
            
            if(inPageYn==1):
                inParam["pageNo"] = i
            queryParams = '?' + urlencode(inParam)
            requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)
            response = requests.get(inUrl+queryParams,verify=False)
            response.encoding=STDENCODING
            rowData = pd.DataFrame()
            if(inType=="jsonabnormal"):
                # 비정상 데이터는 response 섹션이 없음
                if(response.json().get('response') == None):
                    jsondata = response.json()["header"]["resultMsg"]
                    if( jsondata != "NORMAL_SERVICE"):
                        print("SERVER ERROR ",jsondata)
                        break
                    
#               여기서부터는 데이터는 있는데, totalcount>0 경우는 items 정보에 세부정보가 더 있음
               
                jsondata = response.json()["response"]["body"][jsonkey]
                if( jsondata == []):
                    print("{} page is empty".format(i))
                    break
                if( jsonkey == "detail1"):
                    jsondata["index"]=[0]
                rowData = pd.DataFrame(jsondata)
                emptyPd = emptyPd.append(rowData)

                print(">totalCount "+str(response.json()["response"]["body"]["totalCount"]))
                if(0 < int(response.json()["response"]["body"]["totalCount"])):
                    global g_totalCount
                    lock.acquire()
                    if g_totalCount[x] == 0:
                        g_totalCount[x] = int(response.json()["response"]["body"]["totalCount"])
                    lock.release()
                    
            else:
                print("Error")          
     
            if(inPageYn == 0):
                print("{} no pageNo".format(inPageYn))
                break
            i = i+1

        emptyPd.columns = emptyPd.columns.str.lower()
        emptyPd.shape
#         print("dataframe{}, param:{} rows: {} completed".format(inDataName,inParam, emptyPd.shape[1] )     )
        return emptyPd       
    except Exception as e:
            print(e)     

In [32]:
numOfThread = 10

resultDfMerged = list(pd.DataFrame() for x in range(numOfThread))

# 건설자재 품질검사 등록정보 목록 수
g_totalCount = list(range(numOfThread))

In [33]:
def threadReq(startIdx, endIdx,x):
    numOfRows = 1000 # 서버에서 최대 응답 가능한 row임
    resultDf = pd.DataFrame()
    for i in range(startIdx,endIdx):
        pageNo = 1
        print("i=",i)
        print("x=",x)
        year = dateRange[i]
        global g_totalCount
        g_totalCount[x] = 0
        global resultDfMerged
        while True:
            BASEPARAM={"serviceKey":APIKEY,"year": year, "sortField": "tstBgDt","sortOrder": "ASC", "pageNo": pageNo, 'numOfRows':numOfRows, "type":"json"}
            print("BASEPARAM ",BASEPARAM)
            resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY,"jsonabnormal",x)
            if resultDf.empty: # 정상 데이터가 없는 경우
                print("No Data")
                break
            else:
                lock.acquire()
                resultDfMerged[x] = resultDfMerged[x].append(resultDf)
                g_totalCount[x] -= numOfRows
                print("x : ",x," g_totalCount : ", g_totalCount[x], " numOfRows : ", numOfRows)
                if(g_totalCount[x] <= 0):
                    break
                lock.release()
                pageNo += 1

In [34]:
dateRange

[2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022]

In [35]:

resultDfLastMerged = pd.DataFrame()
section = list(range(0, len(dateRange), int(len(dateRange)/numOfThread))) # we have "numåOfThread +1" sections
section

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20]

In [36]:
# dfCombiLen = 6 # for test
if len(dateRange) < numOfThread:
    print("데이터의 갯수가 적어서 쓰레드를 시작할 수 없습니다.")
else:
    x_ls =list(range(numOfThread))
    thread_list = []
    results = []
    idx = 0
    for x in x_ls:
        startIdx = section[x]
        endIdx =  section[x+1] -1 if (x+1 <= numOfThread -1) else len(dateRange) - 1
        print("start ",startIdx, " end ",endIdx)
        print(x)
        mythread = threading.Thread(target=threadReq, args=(startIdx, endIdx+1,idx))
        idx += 1
        thread_list.append(mythread)
    for i in thread_list:
        i.start()
    for i in thread_list:
        i.join()
    for i in range(0,numOfThread):
        resultDfLastMerged = resultDfLastMerged.append(resultDfMerged[i]) 



start  0  end  1
0
start  2  end  3
1
start  4  end  5
2
start  6  end  7
3
start  8  end  9
4
start  10  end  11
5
start  12  end  13
6
start  14  end  15
7
start  16  end  17
8
start  18  end  21
9
i= 0
x= 0
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'year': 2001, 'sortField': 'tstBgDt', 'sortOrder': 'ASC', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
i= 2
x= 1
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'year': 2003, 'sortField': 'tstBgDt', 'sortOrder': 'ASC', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
i= 4
x= 2
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'year': 2005, 'sortField': 'tstBgDt', 'sortOrder': 'ASC', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
i= 6
x= 3
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'year': 2007, 'sortField': 'tstBgDt', 'sortOrder': 'ASC', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page

In [41]:
resultDfLastMerged

,rn,mtlnm,perno,ornm,cwknm,ordrnm,cstrnm,stwrdt,ccwxpcdt,tstbgdt,tsteddt,rpcdisdt,smorsecdnm,smornm,prdcntnnm,smorpcknplacnm,otcuspucts,altsitnm
0,1,한솔검사엔지니어링(주),제2008-12호,서울지방국토관리청,벡스코 시설확충사업 신축공사,None,현대건설(주),None,None,2001-01-17,2001-01-17,2001-01-17,None,None,None,None,None,UT
1,2,한솔검사엔지니어링(주),제2008-12호,서울지방국토관리청,벡스코 시설확충사업 신축공사,None,현대건설(주),None,None,2001-01-17,2001-01-17,2001-01-17,None,None,None,None,None,MT
2,3,(주)다우엔지니어링,제2010-23호,서울지방국토관리청,송우~무봉간 도로 확포장 건설공사,포천시청,일신종합건설㈜,None,None,2001-10-29,2001-11-06,2001-11-06,None,폐콘크리트,None,현장 내(STA. 1+780~1+820),품질시험,None
0,1,(주)한국건설품질기술원,제2013-7호,서울지방국토관리청,파로호 농업농촌테마공원조성사업 조경토목건축공사,한국농어촌공사 홍천춘천지사,대림종합건설(주),None,None,2003-09-16,2003-09-23,2003-09-23,기타 건설자재ㆍ부재,노상,None,No. 5+7.5,현장 검사시험용,현장밀도
0,1,지에스이앤씨(주),제2008-22호,서울지방국토관리청,김해 아울렛 신축공사,신일기초,신일기초,None,None,2008-01-15,2008-03-15,2008-03-15,None,None,None,None,설계지지력 확인,말뚝재하시험
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,11996,(주)한국건설품질기술원,경기1-3-37호,경기도청,수원 농수산물 도매시장시설 현대화사업,수원시,코오롱글로벌(주),2017-11-16,2022-01-04,2021-12-29,2022-01-04,2022-01-04,기타 건설자재ㆍ부재,노상,대한민국,현장내,품질관리용,도로평판재하시험(KS F 2310)
996,11997,건설공인시험연구원 주식회사,경기1-3-88호,경기도청,완주 삼봉 B-1BL 우미건설 신축공사,우미건설(주),우미건설(주),2020-09-09,2022-10-31,2021-12-29,2022-03-08,2022-03-10,기타 건설자재ㆍ부재,도기질타일(WMWB36W01),중국,완주 삼봉 B-1BL 우미건설 신축공사 현장내,품질시험,"겉모양,치수,치수의불규칙도,뒤틀림,꺽임강도,흡수율,내마모성,내균열성"
997,11998,(주)한국건설재료시험연구소,경남-3-5호,경상남도청,명지동 근린상가 신축공사,코리아신탁,우호건설(주),2021-09-15,2023-09-02,2021-12-29,2022-02-03,2022-02-03,기타 건설자재ㆍ부재,SCW 공시체,대한민국,부산광역시 강서구 명지동 3581-1,품질관리,일축압축시험
998,11999,한국에스지에스(주),경기1-3-41호,경기도청,신안산선 복선전철 민간투자사업 6공구 건설공사,넥스트레인㈜,㈜포스코건설 외 13개사,2020-04-10,2025-04-09,2021-12-29,2022-02-07,2022-02-07,기타 건설자재ㆍ부재,수팽창 고무 지수재,대한민국,개착구간,품질관리용,경도 (Type A)


In [42]:
cf.savedata(resultDfLastMerged, SITENAME,DATANAME,SERVICENAME)

건설자재 품질검사 등록정보 save compled


In [43]:
resultDfLastMerged.shape

(1263379, 18)

In [2]:
#임시 
imsiDf = pd.read_csv(\
         "../output/건설사업정보시스템/건설자재 품질검사 등록정보/selectIoPmQtlTsitStsList.csv",low_memory=False, encoding="ms949")
dfCombi = imsiDf.loc[:,["cstrnm"]].drop_duplicates()
dfCombi.reset_index(inplace=True, drop=True)
dfCombiLen = dfCombi.shape[0]


In [3]:
dfCombi

,cstrnm
0,현대건설(주)
1,일신종합건설㈜
2,대림종합건설(주)
3,신일기초
4,만수건설
...,...
74877,정하건설(주)\t\t\t
74878,"(주)태영건설,(주)코메"
74879,가진종합건설(주)
74880,㈜보원건설


In [22]:
mydf = dfCombi.cstrnm =="(주)디제이 종합건설"

In [23]:
dfCombi[mydf]

,cstrnm
60637,(주)디제이 종합건설
